In [2]:
import os
import numpy as np
import pandas as pd
import Utils
import itertools

In [3]:
DATA_PATH = '../data/'
EXPORT_PATH = '/export/'

files = {
    'data': [
        DATA_PATH + 'baskets.parquet',
        DATA_PATH + 'coupon_index.parquet', 
        DATA_PATH + 'coupons.parquet',
        #DATA_PATH + 'prepare.parquet'
   ]
}
cleaned = pd.read_parquet(DATA_PATH + 'clean.parquet')

In [4]:
cleaned

,week,shopper,product,price,discount,purchased
0,0,0,71,629.0,0.00,1
1,0,0,91,605.0,0.00,1
2,0,0,116,715.0,0.00,1
3,0,0,123,483.0,0.00,1
4,0,0,157,592.0,0.00,1
...,...,...,...,...,...,...
110673244,89,1998,101,NaN,0.25,0
110673245,89,1999,207,NaN,0.25,0
110673246,89,1999,246,NaN,0.30,0
110673247,89,1999,51,NaN,0.15,0


In [15]:
aggregation_function = lambda array: array *2

In [17]:
calcu = lambda row: row['product'] * 3

In [10]:
cleaned['price'] = cleaned.apply(calc)

In [11]:
cleaned

,week,shopper,product,price,discount,purchased
0,0,0,71,142.0,0.00,1
1,0,0,91,182.0,0.00,1
2,0,0,116,232.0,0.00,1
3,0,0,123,246.0,0.00,1
4,0,0,157,314.0,0.00,1
...,...,...,...,...,...,...
110673244,89,1998,101,202.0,0.25,0
110673245,89,1999,207,414.0,0.25,0
110673246,89,1999,246,492.0,0.30,0
110673247,89,1999,51,102.0,0.15,0


In [104]:
cleaned["price_new"] = cleaned.apply(lambda row: row.price - row.product)

AttributeError: 'Series' object has no attribute 'price'

In [13]:
columns = range(100)

In [19]:
soso = pd.DataFrame({(column): [1,2,3,4,5,6] for column in columns})

In [ ]:
        n_rows = df[row_name].nunique()
        n_cols = df[column_name].nunique()

        mapping = self._init_df_map(n_rows, n_cols, initial_array)
        print(mapping)
        
        append_to_map = lambda row: mapping.loc[
            int(row[row_name]), str(int(row[column_name]))
        ].append(row[value_name])

        df.progress_apply(append_to_map, axis=1)

In [ ]:
        Might need this later?
        df = pd.DataFrame(index=range(n_rows), columns=range(n_cols))
        for col in df.columns:
            df[col] = df[col].apply(lambda x: initial_array)
        
        '''

In [ ]:
        if initial_array == None:
            initial_array = []

        n_rows = df[row_name].nunique()
        n_cols = df[column_name].nunique()

        mapping = self._init_df_map(n_rows, n_cols, initial_array)
        print(mapping)
        
        append_to_map = lambda row: mapping.loc[
            int(row[row_name]), str(int(row[column_name]))
        ].append(row[value_name])

        df.progress_apply(append_to_map, axis=1)

        return mapping

In [17]:
cleaned_new = cleaned[['week', 'shopper', 'product']]

In [80]:
# _get_mapping
def get_mapping(
    df,
    row_name,
    column_name,
    value_name,
    initial_value: None,
):
    n_rows = df[row_name].nunique()
    n_cols = df[column_name].nunique()
    
    table = pd.DataFrame(itertools.product(list(range(n_rows)), list(range(n_cols))))
    table.columns = [row_name, column_name]
    
    #table[value_name] = [initial_value] * table.shape[0]
    
    def aggregate(x):
        return [initial_value] + list(x) if (initial_value) else list(x)
    
    #table = pd.DataFrame(index=range(n_rows), columns=range(n_cols))
    history = df.groupby([row_name, column_name]).agg({'week':aggregate}).reset_index()
    merged = table.merge(history, how='left')
    merged[value_name] = merged[value_name].apply(lambda d: d if isinstance(d, list) else [initial_value])
    pivot = merged.pivot_table(index=row_name, columns=column_name, values=value_name, aggfunc='first')
    #history = df.groupby([row_name, column_name])[value_name].apply(aggregate).reset_index(name=f'{value_name}_list')
    return pivot

In [81]:
map_config = {
    "df": cleaned,
    "row_name": "shopper",
    "column_name": "product",
    "value_name": "week",
    "initial_value": -1,
}

table = get_mapping(**map_config)

In [96]:
arr = np.array(table.loc[3,3])

In [97]:
arr

array([-1,  5, 20])

In [82]:
table

product,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
shopper,,,,,,,,,,,,,,,,,,,,,
0,[-1],"[-1, 57, 61]","[-1, 44, 45, 88]","[-1, 22, 87]","[-1, 24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]","[-1, 0, 1]",[-1],"[-1, 2, 7, 11]","[-1, 6, 25, 80]","[-1, 35, 41, 48, 51]",...,"[-1, 27, 57, 70]","[-1, 14, 35, 52]","[-1, 9, 48, 49, 80]","[-1, 44, 8, 60]","[-1, 5, 8, 20, 37, 38, 56, 59, 16, 66]","[-1, 63, 87]","[-1, 29]",[-1],[-1],"[-1, 10, 26, 28, 32, 35, 40, 65, 67, 70, 76, 7..."
1,"[-1, 7, 11, 48, 61, 66, 81, 32]","[-1, 2, 6, 18, 20, 27, 29, 49, 53, 62, 70, 73,...","[-1, 13, 14, 81]","[-1, 65]","[-1, 5, 13, 21, 23, 30, 47, 69, 82, 87]","[-1, 42, 85]","[-1, 61]","[-1, 0]","[-1, 54, 88]","[-1, 87]",...,"[-1, 44]","[-1, 71, 88]","[-1, 34, 40, 55, 82]","[-1, 22, 59, 78, 84]","[-1, 45, 77]","[-1, 9, 27, 56, 70, 72]",[-1],"[-1, 4, 28]",[-1],"[-1, 4, 19, 22, 35, 79, 88, 25, 57, 83]"
2,"[-1, 16, 69]","[-1, 2, 3, 6, 7, 8, 9, 11, 13, 16, 18, 19, 21,...","[-1, 26]","[-1, 52]","[-1, 6, 18, 20, 34, 51, 77]","[-1, 25, 68, 82]","[-1, 5, 10, 15, 32, 54, 12, 14, 28]",[-1],"[-1, 55]","[-1, 1, 17, 30, 88]",...,"[-1, 4, 5, 16, 18, 21, 26, 33, 36, 55, 67, 70,...",[-1],"[-1, 45]","[-1, 35, 40, 62]","[-1, 1, 21, 46, 79, 89]","[-1, 75, 77]","[-1, 50, 53, 73, 78]","[-1, 8, 16, 37]","[-1, 3, 14, 20, 42, 45, 50, 51, 54, 62]","[-1, 0, 23, 30, 31, 39]"
3,"[-1, 7]","[-1, 63, 71]","[-1, 41, 73, 83]","[-1, 5, 20]","[-1, 7, 19, 27, 65]","[-1, 79]","[-1, 0, 13, 16, 18, 20, 23, 30, 32, 34, 42, 44...","[-1, 11, 29, 38, 47, 69, 85, 1]","[-1, 26, 54, 66, 85]","[-1, 28, 49]",...,[-1],"[-1, 8, 47, 54]",[-1],"[-1, 57, 45, 59, 72, 74]","[-1, 46, 62, 77]","[-1, 14, 87]","[-1, 9, 48, 70]",[-1],"[-1, 2, 5, 10, 23, 38, 44, 47, 48, 58, 61, 62,...","[-1, 16, 51, 71, 2]"
4,"[-1, 31]","[-1, 2, 34, 44, 67, 79]","[-1, 6, 15, 21, 22, 27, 28, 58, 73, 33, 52]","[-1, 8, 19]","[-1, 38]","[-1, 21, 37, 45]","[-1, 25, 29]","[-1, 9, 32, 56, 65, 70]","[-1, 64, 74]","[-1, 1, 51]",...,"[-1, 57]","[-1, 36]","[-1, 39]","[-1, 53, 72, 77, 85]",[-1],[-1],"[-1, 43, 78]","[-1, 14, 55]","[-1, 0, 17, 19, 22, 44]","[-1, 20, 37]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,"[-1, 42]","[-1, 1, 4, 8, 21, 23, 24, 25, 29, 34, 37, 39, ...",[-1],"[-1, 43, 54]","[-1, 11, 30, 35, 76]","[-1, 39]","[-1, 10, 17, 22, 56, 66, 85, 86, 87]","[-1, 25, 86]","[-1, 26, 63, 65, 83, 89]","[-1, 66]",...,"[-1, 3, 59, 88]","[-1, 29, 44]",[-1],[-1],"[-1, 24, 78]",[-1],"[-1, 9, 70]","[-1, 13, 38]","[-1, 0, 6, 8, 9, 15, 22, 37, 42, 43, 48, 51, 5...","[-1, 6, 23]"
1996,"[-1, 60]","[-1, 19]","[-1, 57, 74]","[-1, 83]",[-1],"[-1, 0, 2, 5, 10, 20, 32, 37, 48, 71, 75, 78, ...","[-1, 70, 86, 89]","[-1, 41, 11]","[-1, 48, 76]","[-1, 54, 72]",...,"[-1, 38, 84]","[-1, 2, 74, 85]","[-1, 12, 19, 86]","[-1, 9, 17, 31]","[-1, 8, 55, 57]","[-1, 22, 58]","[-1, 6, 64]","[-1, 39]",[-1],"[-1, 0, 1, 4, 5, 7, 8, 10, 14, 16, 21, 22, 24,..."
1997,"[-1, 37, 74, 3, 56]","[-1, 17, 76]","[-1, 12]","[-1, 53]","[-1, 81]","[-1, 54, 46, 74]","[-1, 28, 61]","[-1, 10, 49, 51]",[-1],[-1],...,"[-1, 29, 87]","[-1, 15, 62]","[-1, 26, 31, 54, 69]","[-1, 4, 23, 76]","[-1, 36, 65]","[-1, 86]","[-1, 83, 84]",[-1],"[-1, 37, 43]","[-1, 14, 28, 44, 45, 48, 50, 56, 60, 66, 72, 7..."


In [16]:
cleaned_new

,week,shopper,product
0,0,0,71
1,0,0,91
2,0,0,116
3,0,0,123
4,0,0,157
...,...,...,...
110673244,89,1998,101
110673245,89,1999,207
110673246,89,1999,246
110673247,89,1999,51


In [291]:
table

,shopper,product,week
0,0,0,-1
1,0,1,-1
2,0,2,-1
3,0,3,-1
4,0,4,-1
...,...,...,...
499995,1999,245,-1
499996,1999,246,-1
499997,1999,247,-1
499998,1999,248,-1


In [288]:
history

,shopper,product,week
0,0,1,"[-1, 57, 61]"
1,0,2,"[-1, 44, 45, 88]"
2,0,3,"[-1, 22, 87]"
3,0,4,"[-1, 24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]"
4,0,5,"[-1, 0, 1]"
...,...,...,...
441324,1999,245,"[-1, 5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70..."
441325,1999,246,"[-1, 30, 83, 89]"
441326,1999,247,"[-1, 28, 56, 59]"
441327,1999,248,"[-1, 57, 68]"


In [271]:
# table

,shopper,product
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
499995,1999,245
499996,1999,246
499997,1999,247
499998,1999,248


In [268]:
# history

,shopper,product,week
0,0,1,"[-1, 57, 61]"
1,0,2,"[-1, 44, 45, 88]"
2,0,3,"[-1, 22, 87]"
3,0,4,"[-1, 24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]"
4,0,5,"[-1, 0, 1]"
...,...,...,...
441324,1999,245,"[-1, 5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70..."
441325,1999,246,"[-1, 30, 83, 89]"
441326,1999,247,"[-1, 28, 56, 59]"
441327,1999,248,"[-1, 57, 68]"


In [254]:
table

,shopper,product,week
0,0,1,"[57, 61]"
1,0,2,"[44, 45, 88]"
2,0,3,"[22, 87]"
3,0,4,"[24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]"
4,0,5,"[0, 1]"
...,...,...,...
441324,1999,245,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89..."
441325,1999,246,"[30, 83, 89]"
441326,1999,247,"[28, 56, 59]"
441327,1999,248,"[57, 68]"


In [251]:
table

,shopper,product,week
0,0,0,[]
1,0,1,[]
2,0,2,[]
3,0,3,[]
4,0,4,[]
...,...,...,...
499995,1999,245,[]
499996,1999,246,[]
499997,1999,247,[]
499998,1999,248,[]


In [241]:
pivot

,shopper,product,week_list
0,0,1,"[57, 61]"
1,0,2,"[44, 45, 88]"
2,0,3,"[22, 87]"
3,0,4,"[24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]"
4,0,5,"[0, 1]"
...,...,...,...
441324,1999,245,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89..."
441325,1999,246,"[30, 83, 89]"
441326,1999,247,"[28, 56, 59]"
441327,1999,248,"[57, 68]"


In [ ]:
for col in table.columns:
    table[col] = table[col].apply(lambda x: initial_value)

In [90]:
product = list(range(250))
shopper = list(range(2000))
initial_value = []

table = pd.DataFrame(itertools.product(shopper, product))
table.columns = ['shopper', 'product']
table['']
table

,shopper,product
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
499995,1999,245
499996,1999,246
499997,1999,247
499998,1999,248


In [154]:
listi = list([333, 34])
listitwo = listi.append(list([33,33]))
listitwo

In [ ]:
       append_to_map = lambda row: mapping.loc[
            int(row[row_name]), str(int(row[column_name]))
        ].append(row[value_name])

        df.progress_apply(append_to_map, axis=1)

In [160]:
listis = lambda x: [-np.inf] + list(x)

history_new = cleaned.groupby(['shopper', 'product']).agg({'week':listis}).reset_index()
history_new

,shopper,product,week
0,0,1,"[-inf, 57, 61]"
1,0,2,"[-inf, 44, 45, 88]"
2,0,3,"[-inf, 22, 87]"
3,0,4,"[-inf, 24, 40, 45, 51, 54, 58, 60, 70, 12, 17,..."
4,0,5,"[-inf, 0, 1]"
...,...,...,...
441324,1999,245,"[-inf, 5, 17, 22, 24, 25, 33, 49, 60, 63, 67, ..."
441325,1999,246,"[-inf, 30, 83, 89]"
441326,1999,247,"[-inf, 28, 56, 59]"
441327,1999,248,"[-inf, 57, 68]"


In [116]:
history = cleaned.groupby(['shopper', 'product'])['week'].apply(list).reset_index(name='weeks')
history

In [117]:
history

,shopper,product,weeks
0,0,1,"[57, 61]"
1,0,2,"[44, 45, 88]"
2,0,3,"[22, 87]"
3,0,4,"[24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]"
4,0,5,"[0, 1]"
...,...,...,...
441324,1999,245,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89..."
441325,1999,246,"[30, 83, 89]"
441326,1999,247,"[28, 56, 59]"
441327,1999,248,"[57, 68]"


In [93]:
merged = table.merge(history, how='left')
merged
keep it simple, we don't really need to pivot the table
pivot = merged.pivot_table(index='shopper', columns='product', values='weeks', aggfunc='first')


,shopper,product,weeks
0,0,0,NaN
1,0,1,"[57, 61]"
2,0,2,"[44, 45, 88]"
3,0,3,"[22, 87]"
4,0,4,"[24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]"
...,...,...,...
499995,1999,245,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89..."
499996,1999,246,"[30, 83, 89]"
499997,1999,247,"[28, 56, 59]"
499998,1999,248,"[57, 68]"


In [98]:
merged
#df.pivot(index='car_model', columns='date', values='0-60mph_in_seconds')

NameError: name 'merged' is not defined

In [87]:
#merged.set_index(['shopper','product', 'weeks']).unstack('shopper')

TypeError: unhashable type: 'list'

In [88]:
# 

product,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
shopper,,,,,,,,,,,,,,,,,,,,,
0,NaN,"[57, 61]","[44, 45, 88]","[22, 87]","[24, 40, 45, 51, 54, 58, 60, 70, 12, 17, 38]","[0, 1]",NaN,"[2, 7, 11]","[6, 25, 80]","[35, 41, 48, 51]",...,"[27, 57, 70]","[14, 35, 52]","[9, 48, 49, 80]","[44, 8, 60]","[5, 8, 20, 37, 38, 56, 59, 16, 66]","[63, 87]",[29],NaN,NaN,"[10, 26, 28, 32, 35, 40, 65, 67, 70, 76, 78, 8..."
1,"[7, 11, 48, 61, 66, 81, 32]","[2, 6, 18, 20, 27, 29, 49, 53, 62, 70, 73, 76,...","[13, 14, 81]",[65],"[5, 13, 21, 23, 30, 47, 69, 82, 87]","[42, 85]",[61],[0],"[54, 88]",[87],...,[44],"[71, 88]","[34, 40, 55, 82]","[22, 59, 78, 84]","[45, 77]","[9, 27, 56, 70, 72]",NaN,"[4, 28]",NaN,"[4, 19, 22, 35, 79, 88, 25, 57, 83]"
2,"[16, 69]","[2, 3, 6, 7, 8, 9, 11, 13, 16, 18, 19, 21, 23,...",[26],[52],"[6, 18, 20, 34, 51, 77]","[25, 68, 82]","[5, 10, 15, 32, 54, 12, 14, 28]",NaN,[55],"[1, 17, 30, 88]",...,"[4, 5, 16, 18, 21, 26, 33, 36, 55, 67, 70, 81,...",NaN,[45],"[35, 40, 62]","[1, 21, 46, 79, 89]","[75, 77]","[50, 53, 73, 78]","[8, 16, 37]","[3, 14, 20, 42, 45, 50, 51, 54, 62]","[0, 23, 30, 31, 39]"
3,[7],"[63, 71]","[41, 73, 83]","[5, 20]","[7, 19, 27, 65]",[79],"[0, 13, 16, 18, 20, 23, 30, 32, 34, 42, 44, 45...","[11, 29, 38, 47, 69, 85, 1]","[26, 54, 66, 85]","[28, 49]",...,NaN,"[8, 47, 54]",NaN,"[57, 45, 59, 72, 74]","[46, 62, 77]","[14, 87]","[9, 48, 70]",NaN,"[2, 5, 10, 23, 38, 44, 47, 48, 58, 61, 62, 83]","[16, 51, 71, 2]"
4,[31],"[2, 34, 44, 67, 79]","[6, 15, 21, 22, 27, 28, 58, 73, 33, 52]","[8, 19]",[38],"[21, 37, 45]","[25, 29]","[9, 32, 56, 65, 70]","[64, 74]","[1, 51]",...,[57],[36],[39],"[53, 72, 77, 85]",NaN,NaN,"[43, 78]","[14, 55]","[0, 17, 19, 22, 44]","[20, 37]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,[42],"[1, 4, 8, 21, 23, 24, 25, 29, 34, 37, 39, 41, ...",NaN,"[43, 54]","[11, 30, 35, 76]",[39],"[10, 17, 22, 56, 66, 85, 86, 87]","[25, 86]","[26, 63, 65, 83, 89]",[66],...,"[3, 59, 88]","[29, 44]",NaN,NaN,"[24, 78]",NaN,"[9, 70]","[13, 38]","[0, 6, 8, 9, 15, 22, 37, 42, 43, 48, 51, 52, 6...","[6, 23]"
1996,[60],[19],"[57, 74]",[83],NaN,"[0, 2, 5, 10, 20, 32, 37, 48, 71, 75, 78, 80, ...","[70, 86, 89]","[41, 11]","[48, 76]","[54, 72]",...,"[38, 84]","[2, 74, 85]","[12, 19, 86]","[9, 17, 31]","[8, 55, 57]","[22, 58]","[6, 64]",[39],NaN,"[0, 1, 4, 5, 7, 8, 10, 14, 16, 21, 22, 24, 27,..."
1997,"[37, 74, 3, 56]","[17, 76]",[12],[53],[81],"[54, 46, 74]","[28, 61]","[10, 49, 51]",NaN,NaN,...,"[29, 87]","[15, 62]","[26, 31, 54, 69]","[4, 23, 76]","[36, 65]",[86],"[83, 84]",NaN,"[37, 43]","[14, 28, 44, 45, 48, 50, 56, 60, 66, 72, 74, 8..."


In [ ]:
merged[merged[]]

In [ ]:
def get_history(shopper, product):
    

In [75]:
    def get_history(
        self, 
        shopper: int, 
        product: int, 
        week: int, 
        mapping: str = "product_histories"
    ):
        arr = np.array(self.mappings[mapping].loc[shopper, str(product)])
        return arr[arr < week]


TypeError: unhashable type: 'list'

In [ ]:
n_shoppers = 2000
n_products = 250
initial_value = []

table = pd.DataFrame(index=range(n_shoppers), columns=range(n_products))

for col in table.columns:
    table[col] = table[col].apply(lambda x: initial_value)
    
table

In [ ]:
[ 
    [   # shopper 0
        [0, 10, 56],     # weeks for product 0
        [4, 45, 78, 13], # weeks for product 1
        ...
        ...
        ...
        [7, 12, 56]      # weeks for product 249
    ],
    [   # shopper 1
        [67, 70],        # weeks for product 0
        [3, 43],         # weeks for product 1
        ...
    ],
    [   # shopper 2
        [-inf],          # weeks for product 0
        [45, 78],        # weeks for product 1
        ...
    ],
    ...
    ...
    ...
    [   # shopper 1999
        [-inf],          # weeks for product 0
        [0, 14, 78],     # weeks for product 1
        ...
    ]
]